<a href="https://colab.research.google.com/github/SB-1996122/stock-market/blob/main/Stock_Market_Performance_Analysis_using_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install yfinance


In [4]:
#collecting real-time stock market data using the yfinance API
import pandas as pd
import yfinance as yf
from datetime import datetime

start_date = datetime.now() - pd.DateOffset(months=3)
end_date = datetime.now()

tickers = ['AAPL', 'MSFT', 'NFLX', 'GOOG']

df_list = []

for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date)
    df_list.append(data)

df = pd.concat(df_list, keys=tickers, names=['Ticker', 'Date'])
print(df.head())



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
                         Open        High         Low       Close   Adj Close  \
Ticker Date                                                                     
AAPL   2023-06-13  182.800003  184.149994  182.440002  183.309998  183.062790   
       2023-06-14  183.369995  184.389999  182.020004  183.949997  183.701935   
       2023-06-15  183.960007  186.520004  183.779999  186.009995  185.759155   
       2023-06-16  186.729996  186.990005  184.270004  184.919998  184.670624   
       2023-06-20  184.410004  186.100006  184.410004  185.009995  184.760498   

                      Volume  
Ticker Date                   
AAPL   2023-06-13   54929100  
       2023-06-14   57462900  
       2023-06-15   65433200  
 

In [6]:
df = df.reset_index()
print(df.head())

   index Ticker       Date        Open        High         Low       Close  \
0      0   AAPL 2023-06-13  182.800003  184.149994  182.440002  183.309998   
1      1   AAPL 2023-06-14  183.369995  184.389999  182.020004  183.949997   
2      2   AAPL 2023-06-15  183.960007  186.520004  183.779999  186.009995   
3      3   AAPL 2023-06-16  186.729996  186.990005  184.270004  184.919998   
4      4   AAPL 2023-06-20  184.410004  186.100006  184.410004  185.009995   

    Adj Close     Volume  
0  183.062790   54929100  
1  183.701935   57462900  
2  185.759155   65433200  
3  184.670624  101235600  
4  184.760498   49799100  


In [7]:
import plotly.express as px
fig = px.line(df, x='Date',
              y='Close',
              color='Ticker',
              title="Stock Market Performance for the Last 3 Months")
fig.show()

In [8]:
fig = px.area(df, x='Date', y='Close', color='Ticker',
              facet_col='Ticker',
              labels={'Date':'Date', 'Close':'Closing Price', 'Ticker':'Company'},
              title='Stock Prices for Apple, Microsoft, Netflix, and Google')
fig.show()

In [9]:
df['MA10'] = df.groupby('Ticker')['Close'].rolling(window=10).mean().reset_index(0, drop=True)
df['MA20'] = df.groupby('Ticker')['Close'].rolling(window=20).mean().reset_index(0, drop=True)

for ticker, group in df.groupby('Ticker'):
    print(f'Moving Averages for {ticker}')
    print(group[['MA10', 'MA20']])

Moving Averages for AAPL
          MA10        MA20
0          NaN         NaN
1          NaN         NaN
2          NaN         NaN
3          NaN         NaN
4          NaN         NaN
..         ...         ...
58  183.800999  180.350000
59  183.445000  180.318500
60  183.624998  180.328999
61  183.699998  180.407500
62  183.310999  180.249500

[63 rows x 2 columns]
Moving Averages for GOOG
           MA10        MA20
189         NaN         NaN
190         NaN         NaN
191         NaN         NaN
192         NaN         NaN
193         NaN         NaN
..          ...         ...
247  134.476001  132.184501
248  134.775000  132.487001
249  135.453000  132.836501
250  136.158000  133.215001
251  136.586002  133.427001

[63 rows x 2 columns]
Moving Averages for MSFT
           MA10        MA20
63          NaN         NaN
64          NaN         NaN
65          NaN         NaN
66          NaN         NaN
67          NaN         NaN
..          ...         ...
121  327.370004  324.19

In [10]:
for ticker, group in df.groupby('Ticker'):
    fig = px.line(group, x='Date', y=['Close', 'MA10', 'MA20'],
                  title=f"{ticker} Moving Averages")
    fig.show()

In [11]:
df['Volatility'] = df.groupby('Ticker')['Close'].pct_change().rolling(window=10).std().reset_index(0, drop=True)
fig = px.line(df, x='Date', y='Volatility',
              color='Ticker',
              title='Volatility of All Companies')
fig.show()

In [12]:
# create a DataFrame with the stock prices of Apple and Microsoft
apple = df.loc[df['Ticker'] == 'AAPL', ['Date', 'Close']].rename(columns={'Close': 'AAPL'})
microsoft = df.loc[df['Ticker'] == 'MSFT', ['Date', 'Close']].rename(columns={'Close': 'MSFT'})
df_corr = pd.merge(apple, microsoft, on='Date')

# create a scatter plot to visualize the correlation
fig = px.scatter(df_corr, x='AAPL', y='MSFT',
                 trendline='ols',
                 title='Correlation between Apple and Microsoft')
fig.show()